In [1]:
import random
import numpy as np

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## Data loading and preprocessing

In [2]:
import pandas as pd

columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
df = pd.read_csv("data/adult.csv", names=columns, na_values=" ?", header=0)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
df.dropna(inplace=True)

# Zielvariable in boolschen Wert umwandeln
df["income"] = df["income"].apply(lambda x: 1 if x == ">50K" else 0)

In [4]:
from sklearn.model_selection import train_test_split


X = df.drop("income", axis=1)
y = df["income"]

# Daten in Trainings- und Testset aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [5]:
categorical_features = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), X_train.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

## Define Optuna Objectives

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score


def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'class_weight': 'balanced'
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(**params, random_state=42))
    ])
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

In [8]:
from xgboost import XGBClassifier

def objective_xgb(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'eval_metric': 'logloss'
    }
    
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(**params, random_state=42, use_label_encoder=False))
    ])
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred)

## Create Optuna Studies

In [9]:
import optuna

TRIALS = 2

# Random Forest Optimierung
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=TRIALS, show_progress_bar=True)

# XGBoost Optimierung
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=TRIALS, show_progress_bar=True)

[I 2025-03-15 09:32:25,901] A new study created in memory with name: no-name-15a4f21d-6117-42ac-8d74-65ad7ffc486c


  0%|          | 0/2 [00:00<?, ?it/s]

[I 2025-03-15 09:32:28,936] Trial 0 finished with value: 0.6184947958366693 and parameters: {'n_estimators': 232, 'max_depth': 5, 'min_samples_split': 6}. Best is trial 0 with value: 0.6184947958366693.


[I 2025-03-15 09:32:35,516] A new study created in memory with name: no-name-0727639e-e19a-495b-88c3-2323ed8029b9


[I 2025-03-15 09:32:35,513] Trial 1 finished with value: 0.6632718014003819 and parameters: {'n_estimators': 209, 'max_depth': 9, 'min_samples_split': 6}. Best is trial 1 with value: 0.6632718014003819.


  0%|          | 0/2 [00:00<?, ?it/s]

/Users/christopherkeibel/Code/fhswf/mlops/training_server/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [09:32:35] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[I 2025-03-15 09:32:36,140] Trial 0 finished with value: 0.6898612593383138 and parameters: {'learning_rate': 0.0797461743161529, 'max_depth': 8, 'subsample': 0.7281392140862799, 'colsample_bytree': 0.8957263211797086, 'n_estimators': 155, 'scale_pos_weight': 6.903302162938512}. Best is trial 0 with value: 0.6898612593383138.


/Users/christopherkeibel/Code/fhswf/mlops/training_server/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [09:32:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[I 2025-03-15 09:32:36,831] Trial 1 finished with value: 0.717008099094807 and parameters: {'learning_rate': 0.19895775151013562, 'max_depth': 6, 'subsample': 0.6001207435564574, 'colsample_bytree': 0.6256564147093923, 'n_estimators': 286, 'scale_pos_weight': 3.9819533399238223}. Best is trial 1 with value: 0.717008099094807.


In [14]:
import mlflow

mlflow.set_experiment("Income_Classification")


<Experiment: artifact_location='file:///Users/christopherkeibel/Code/fhswf/mlops/training_server/notebooks/mlruns/492963098128935861', creation_time=1742027612318, experiment_id='492963098128935861', last_update_time=1742027612318, lifecycle_stage='active', name='Income_Classification', tags={}>

In [15]:
from mlflow.models import infer_signature
from mlflow.client import MlflowClient


with mlflow.start_run(run_name="random_forest"):
    best_rf = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            **study_rf.best_params,
            class_weight='balanced',
            random_state=42
        ))
    ])
    
    best_rf.fit(X_train, y_train)
    rf_pred = best_rf.predict(X_test)
    
    mlflow.log_params(study_rf.best_params)
    mlflow.log_metrics({
        'accuracy': accuracy_score(y_test, rf_pred),
        'f1_score': f1_score(y_test, rf_pred)
    })
    
    signature = infer_signature(X_train, best_rf.predict(X_train))
    mlflow.sklearn.log_model(
        best_rf,
        "random_forest_model",
        signature=signature,
        input_example=X_train[:5].to_dict(orient='records')
    )
    
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/random_forest_model"
    model_version = mlflow.register_model(model_uri, "random_forest_pipeline")
    client = MlflowClient()
    client.set_registered_model_alias("random_forest_pipeline", "best", model_version.version)

/Users/christopherkeibel/Code/fhswf/mlops/training_server/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'random_forest_pipeline' already exists. Creating a new version of this model...
Created version '6' of model 'random_forest_pipeline'.


In [16]:
with mlflow.start_run(run_name="xgboost"):
    best_xgb = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(
            **study_xgb.best_params,
            random_state=42,
            use_label_encoder=False
        ))
    ])
    
    best_xgb.fit(X_train, y_train)
    xgb_pred = best_xgb.predict(X_test)
    
    mlflow.log_params(study_xgb.best_params)
    mlflow.log_metrics({
        'accuracy': accuracy_score(y_test, xgb_pred),
        'f1_score': f1_score(y_test, xgb_pred)
    })
    
    signature = infer_signature(X_train, best_xgb.predict(X_train))
    mlflow.sklearn.log_model(
        best_xgb,
        "xgb_model",
        signature=signature,
        input_example=X_train[:5].to_dict(orient='records')
    )
    
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/xgb_model"
    model_version = mlflow.register_model(model_uri, "xgb_pipeline")
    client = MlflowClient()
    client.set_registered_model_alias("xgb_pipeline", "best", model_version.version)

/Users/christopherkeibel/Code/fhswf/mlops/training_server/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [09:36:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/christopherkeibel/Code/fhswf/mlops/training_server/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/l